In [ ]:
import sqlite3;
import pandas as pd;

#CONECTAR BASE DE DATOS

conexion = sqlite3.connect('datos_sensores.db');

query = """
SELECT
datos_basicos.tamaño,
datos_basicos.categoria,
orion.sensor_orion,
vega.sensor_vega,
polaris.sensor_polaris,
antares.sensor_antares,




CASE
  WHEN clasificacion.etiqueta = "Positivo" THEN 1
  ELSE 0
END AS etiqueta_numerica

FROM datos_basicos
INNER JOIN orion ON datos_basicos.id = orion.id
INNER JOIN vega ON datos_basicos.id = vega.id
INNER JOIN polaris ON datos_basicos.id = polaris.id
INNER JOIN antares ON datos_basicos.id = antares.id
INNER JOIN clasificacion ON datos_basicos.id = clasificacion.id
LIMIT 300000
"""

dataset = pd.read_sql_query(query,conexion);
dataset.head()

,tamaño,categoria,sensor_orion,sensor_vega,sensor_polaris,sensor_antares,etiqueta_numerica
0,0.02,3,19.77,42.15,35.36,-62.73,1
1,0.11,3,-23.51,-20.64,69.78,-38.02,1
2,0.85,1,86.63,75.26,30.82,-71.62,0
3,0.53,1,-44.91,4.11,71.92,27.17,0
4,0.14,2,-66.34,-48.04,-2.91,-74.93,0


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix;
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# ... (código anterior para la conexión y carga de datos) ...

X = dataset.drop('etiqueta_numerica', axis=1)
Y = dataset['etiqueta_numerica']

X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=0.2, random_state=10)

# Normalización de datos
scaler = StandardScaler()
X_Train_scaled = scaler.fit_transform(X_Train)
X_Test_scaled = scaler.transform(X_Test)

# Manejo de desequilibrio de clases con SMOTE
smote = SMOTE(random_state=1)
X_Train_resampled, Y_Train_resampled = smote.fit_resample(X_Train_scaled, Y_Train)

# Modelo RandomForest
modelo_rf = RandomForestClassifier( n_estimators=100,  # Puedes ajustar este valor
    max_depth=50,    # Puedes ajustar este valor
    min_samples_split=2,  # Puedes ajustar este valor
    min_samples_leaf=1,   # Puedes ajustar este valor
    max_features='auto',  # Puedes ajustar este valor
    random_state=1)
modelo_rf.fit(X_Train_resampled, Y_Train_resampled)

# Predicciones y evaluación del modelo
predicciones_rf = modelo_rf.predict(X_Test_scaled)

accuracy_rf = accuracy_score(Y_Test, predicciones_rf)
print(f"Precisión del Modelo RandomForest: {accuracy_rf:.2f}")

Reporte_rf = classification_report(Y_Test, predicciones_rf)
print(Reporte_rf)

matriz = confusion_matrix(Y_Test,predicciones_rf);
print(matriz)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Precisión del Modelo RandomForest: 0.90
              precision    recall  f1-score   support

           0       0.95      0.85      0.90     30247
           1       0.86      0.96      0.91     29753

    accuracy                           0.90     60000
   macro avg       0.91      0.90      0.90     60000
weighted avg       0.91      0.90      0.90     60000

[[25681  4566]
 [ 1308 28445]]


In [ ]:
conexion2 = sqlite3.connect('datos_sensores.db');

query2 = """
SELECT
datos_basicos.tamaño,
datos_basicos.categoria,
orion.sensor_orion,
vega.sensor_vega,
polaris.sensor_polaris,
antares.sensor_antares,



CASE
  WHEN clasificacion.etiqueta = "Positivo" THEN 1
  WHEN clasificacion.etiqueta = "Negativo" THEN 0
  ELSE NULL
END AS etiqueta_numerica

FROM datos_basicos

INNER JOIN orion ON datos_basicos.id = orion.id
INNER JOIN vega ON datos_basicos.id = vega.id
INNER JOIN polaris ON datos_basicos.id = polaris.id
INNER JOIN antares ON datos_basicos.id = antares.id
INNER JOIN clasificacion ON datos_basicos.id = clasificacion.id

ORDER BY etiqueta_numerica ASC
Limit 100
"""

dataset2 = pd.read_sql_query(query2,conexion2);
dataset2

,tamaño,categoria,sensor_orion,sensor_vega,sensor_polaris,sensor_antares,etiqueta_numerica
0,0.10,2,-91.99,90.35,12.85,45.19,None
1,0.93,2,2.98,14.86,-22.14,45.21,None
2,0.86,3,-22.53,10.25,74.24,1.93,None
3,0.86,1,24.14,-31.35,-67.50,29.67,None
4,0.92,3,16.14,98.17,-14.76,10.59,None
...,...,...,...,...,...,...,...
95,0.61,3,75.72,-65.18,75.00,-33.67,None
96,0.08,1,-3.33,-28.75,-97.28,30.82,None
97,0.41,3,-48.04,-23.84,32.14,-46.10,None
98,0.71,1,15.46,-39.97,45.71,64.84,None


In [ ]:
X2 = dataset2.drop('etiqueta_numerica', axis=1)
Y2 = dataset2['etiqueta_numerica']

# Normalización de datos
scaler2 = StandardScaler()
X_Test_scaled2 = scaler.transform(X2)


# Predicciones y evaluación del modelo
predicciones_rf2 = modelo_rf.predict(X_Test_scaled2)

predicciones_rf2


array([0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1])